In [9]:
import mlflow
import os
import pandas as pd

from mlflow.tracking import MlflowClient
from mlflow.models import infer_signature
from mlflow.store.artifact.artifact_repository_registry import get_artifact_repository

from sklearn.datasets import fetch_california_housing
from sklearn.ensemble import RandomForestRegressor, HistGradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

In [10]:
import datetime
exp_name = f'exp_{datetime.datetime.now().strftime("%d%m%y_%H-%M-%S_%f")}'

In [11]:
exp_id = mlflow.create_experiment(name=exp_name)

In [12]:
housing = fetch_california_housing(as_frame=True)

In [13]:
X_train, X_test, y_train, y_test = train_test_split(housing['data'], housing['target'])
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5)

In [14]:
models = dict(zip(["RandomForest", "LinearRegression", "HistGB"], 
                  [RandomForestRegressor(), LinearRegression(), HistGradientBoostingRegressor()]))

In [15]:
time_mark = f'{datetime.datetime.now().strftime("%d%m%y_%H-%M-%S_%f")}'

with mlflow.start_run(run_name=f"California_run_{time_mark}", experiment_id = exp_id, description = "parent") as parent_run:
    for model_name in models.keys():
        with mlflow.start_run(run_name=model_name, experiment_id=exp_id, nested=True) as child_run:
            model = models[model_name]
            
            model.fit(pd.DataFrame(X_train), y_train)
        
            prediction = model.predict(X_val)
        
            eval_df = X_val.copy()
            eval_df["target"] = y_val
            eval_df["prediction"] = prediction
        
            #signature = infer_signature(X_test, prediction)
            #model_info = mlflow.sklearn.log_model(model, "logreg", signature=signature)
            mlflow.evaluate(
                #model=model_info.model_uri,
                data=eval_df,
                targets="target",
                predictions="prediction",
                model_type="regressor",
                evaluators=["default"],
            )

2025/02/13 21:35:45 INFO mlflow.models.evaluation.default_evaluator: Testing metrics on first row...
2025/02/13 21:35:45 WARNING mlflow.models.evaluation.default_evaluator: SHAP or matplotlib package is not installed, so model explainability insights will not be logged.
2025/02/13 21:35:45 INFO mlflow.tracking._tracking_service.client: 🏃 View run RandomForest at: http://mlflow-service:5000/#/experiments/412476594189829162/runs/dcc6ced3adbc4744ae7791bedac38a39.
2025/02/13 21:35:45 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://mlflow-service:5000/#/experiments/412476594189829162.
2025/02/13 21:35:45 INFO mlflow.models.evaluation.default_evaluator: Testing metrics on first row...
2025/02/13 21:35:45 WARNING mlflow.models.evaluation.default_evaluator: SHAP or matplotlib package is not installed, so model explainability insights will not be logged.
2025/02/13 21:35:45 INFO mlflow.tracking._tracking_service.client: 🏃 View run LinearRegression at: http://mlflow-s